In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import pickle
import seaborn as sns
import re
import datetime


# Notebook options
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 99999
#pd.options.display.float_format = '{:20,.3f}'.format
import random
%matplotlib inline

In [2]:
#%time data = pd.read_csv('archive/Loan_status_2007-2020Q3.gzip.zip', compression="zip",index_col=0)

In [3]:
#%time data.to_pickle("data.pkl")

In [4]:
%time data = pd.read_pickle("data.pkl")

CPU times: user 3.14 s, sys: 2.09 s, total: 5.23 s
Wall time: 5.24 s


In [5]:
column_missing_rate = 100*(data.isnull().sum(axis = 0) / len(data))

In [6]:
100*(data.earliest_cr_line.isna().sum(axis = 0) / len(data))


0.0010254681860459075

In [7]:
column_missing_rate = pd.DataFrame(column_missing_rate)

In [8]:
column_missing_rate = column_missing_rate.reset_index()

In [9]:
column_missing_rate.columns = ['name','rate']

In [10]:
columns_to_delete = []
for index, row in column_missing_rate.iterrows():
    if row['rate'] >= 25:
        columns_to_delete.append(row['name'])
print(columns_to_delete)

['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'orig_projected_additional_accrued_inte

In [11]:
columns_to_delete.sort()

### Below are columns/attributes that has over 25% missing data in original dataset
__We decided to delete them from analysis__

In [12]:
print(columns_to_delete)

['all_util', 'annual_inc_joint', 'deferral_term', 'dti_joint', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'il_util', 'inq_fi', 'inq_last_12m', 'max_bal_bc', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'mths_since_rcnt_il', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'next_pymnt_d', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'open_rv_12m', 'open_rv_24m', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'revol_bal_joint', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_num_rev_accts', 'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_

__Per Group discussion keep sec_app_fico_range_low and sec_app_fico_range_high__

In [13]:
columns_to_delete.remove('sec_app_fico_range_low')

In [14]:
columns_to_delete.remove('sec_app_fico_range_high')

In [15]:
data.drop(columns_to_delete,axis=1,inplace=True)

create a list of variables to delete per discussion

In [16]:
group_agreed_deletation = ['id','sub_grade','initial_list_status','last_credit_pull_d',
                           'mths_since_recent_inq','num_accts_ever_120_pd',
                          'num_bc_sats','num_bc_tl','num_op_rev_tl','num_rev_accts',
                          'num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m',
                          'num_tl_30dpd','num_tl_op_past_12m','total_pymnt',
                          'total_pymnt_inv','url','zip_code','emp_title']

In [17]:
data.drop(group_agreed_deletation,axis=1,inplace=True)

In [18]:
nick_suggested_deletation = ['avg_cur_bal','bc_open_to_buy','collection_recovery_fee',
                             'funded_amnt','funded_amnt_inv','installment','num_actv_bc_tl',
                            'out_prncp_inv','percent_bc_gt_75','policy_code','pub_rec',
                             'tot_hi_cred_lim','total_acc','total_il_high_credit_limit',
                            'total_rec_int','total_rev_hi_lim','pymnt_plan','title']

In [19]:
data.drop(nick_suggested_deletation,axis=1,inplace=True)

In [20]:
data['fico_range_high'][data['fico_range_high']<=600]

Series([], Name: fico_range_high, dtype: float64)

__Dropping observation at index 39786 - NA for all attributes__

In [21]:
data.drop(index=39786,inplace=True)

In [22]:
data['fico'] = (data['fico_range_high']+data['fico_range_low'])/2
data['last_fico'] = data['last_fico_range_high']-2
data['last_fico'] = np.where(data['last_fico']<=0,data['fico'],data['last_fico'])
data['sec_app_fico'] = (data['sec_app_fico_range_high']+data['sec_app_fico_range_low'])/2
data['overall_fico'] = data[['fico', 'sec_app_fico']].mean(axis=1)
fico_to_drop = ['fico','fico_range_high','fico_range_low',
                'last_fico_range_high','last_fico_range_low',
                'sec_app_fico_range_high','sec_app_fico_range_low','sec_app_fico']
data.drop(fico_to_drop,axis=1,inplace=True)

### David's Append

In [23]:
def str_to_rate(x: str):
    if isinstance(x, str):
        x = x[:-1]
        return float(x)
    elif np.isnan(x):
        return x
    else:
        pass
    
def remove_pattern(pattern: str, original: str):
    if isinstance(original, str):
        original = original.lower()
        return int(re.sub(pattern, '', original ))
    else:
        return np.nan
        
def str_date(x: str):
    if isinstance(x, str):
        x = datetime.datetime.strptime(x, '%b-%Y')
    else:
        x= np.nan
    return x

def credit_days(date2: datetime.date, date1: datetime.datetime.date):
    if isinstance(date2, datetime.date) and isinstance(date1, datetime.date):
        difference = date2 - date1
        return int(difference.days)
    else:
        return np.nan

def build_hist(data: pd.DataFrame, x: str):
    fig = px.histogram(data, x=x)
    fig.show()
    
def hist_by_group(data: pd.DataFrame,x: str, grp: str):
    fig = px.histogram(data, x=x,
        color = grp)
    fig.show()

In [24]:
# convert int rate string to float
data["int_rate"] = data["int_rate"].apply(lambda x: str_to_rate(x) )

# convert revol_util string to float
data["revol_util"] = data["revol_util"].apply(lambda x: str_to_rate(x) )

# remove term from term feature and convert to int  - we might  want to hot encode this - Per group discussion leave as original
# data["term"] = data["term"].apply(lambda x: remove_pattern(r"months?\b", x) )

# remove years from emp_length feature and convert to int -  we might  want to hot encode this
#data["emp_length"] = data["emp_length"].apply(lambda x: remove_pattern(r"years?\b", x) )

In [25]:
# convert issue_d string to datetime
data["issue_d"] = data["issue_d"].apply(lambda x: str_date(x) )

# convert earliest_cr_line string to datetime
data["earliest_cr_line"] = data["earliest_cr_line"].apply(lambda x: str_date(x) )

# convert last_pymnt_d string to datetime
data["last_pymnt_d"] = data["last_pymnt_d"].apply(lambda x: str_date(x) )

In [26]:
#  Drop nan from earliest_cr_line  to ensure the credit_days function runs properly
#crates new feature called days_with_Credit from issue_d and earliest_cr_line
data = data.dropna(subset=["earliest_cr_line"], axis=0)
data['days_with_Credit'] = data[['issue_d','earliest_cr_line']].apply(lambda x: credit_days(*x), axis=1)

In [27]:
data = data[data['issue_d']>= "2015-01-01"]

In [28]:
sample_100k = data.sample(n=100000,random_state=101)

In [29]:
# Creating good_loan_or_not column - consolidating loan_status column
good_loan_status = ['Fully Paid','Current','In Grace Period','Issued']
bad_loan_status = ['Charged Off','Late (31-120 days)','Late (16-30 days)','Default']
#'Fully Paid' in good_loan_status
def loan_flag(loan_st: str):
    if loan_st in good_loan_status:
        x = 1
        return x
    else:
        x = 0
        return x
# create good_loan_or_not column 
sample_100k["good_loan_or_not"] = sample_100k["loan_status"].apply(lambda x: loan_flag(x))

In [30]:
sample_100k.to_pickle("loan_100k_sample.pkl")

In [31]:
%time data_100k = pd.read_pickle("loan_100k_sample.pkl")

CPU times: user 30.2 ms, sys: 23.4 ms, total: 53.6 ms
Wall time: 52.6 ms


In [32]:
data_100k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 36800 to 46436
Data columns (total 54 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   loan_amnt                   100000 non-null  float64       
 1   term                        100000 non-null  object        
 2   int_rate                    100000 non-null  float64       
 3   grade                       100000 non-null  object        
 4   emp_length                  92532 non-null   object        
 5   home_ownership              100000 non-null  object        
 6   annual_inc                  100000 non-null  float64       
 7   verification_status         100000 non-null  object        
 8   issue_d                     100000 non-null  datetime64[ns]
 9   loan_status                 100000 non-null  object        
 10  purpose                     100000 non-null  object        
 11  addr_state                  100000 n

In [33]:
100*(data_100k.isnull().sum(axis = 0) / len(data))

loan_amnt                     0.000000
term                          0.000000
int_rate                      0.000000
grade                         0.000000
emp_length                    0.303685
home_ownership                0.000000
annual_inc                    0.000000
verification_status           0.000000
issue_d                       0.000000
loan_status                   0.000000
purpose                       0.000000
addr_state                    0.000000
dti                           0.004595
delinq_2yrs                   0.000000
earliest_cr_line              0.000000
inq_last_6mths                0.000041
open_acc                      0.000000
revol_bal                     0.000000
revol_util                    0.003253
out_prncp                     0.000000
total_rec_prncp               0.000000
total_rec_late_fee            0.000000
recoveries                    0.000000
last_pymnt_d                  0.008255
last_pymnt_amnt               0.000000
collections_12_mths_ex_me